In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20007", "프리미엄"),       # 예시: 침대 카테고리
    ("20008", "스탠다드")         # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_전체공사_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 2개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 프리미엄 (ID: 20007)
-> [프리미엄] 1 페이지 로딩 중...
✅ [프리미엄] 1 페이지 완료. 수집된 ID 6개, 새로운 ID 6개 추가.
-> [프리미엄] 2 페이지 로딩 중...
❌ [프리미엄] 2 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandleVerifier [0x0x7ff77cd5b119+117673]
	GetHandleVerifier [0x0x7ff77cd410a8

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20010", "유로"),       # 예시: 침대 카테고리
    ("20011", "밀란"),
    ("20484", "상판"),
    ("20487", "상품기기")         # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_키친_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 4개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 유로 (ID: 20010)
-> [유로] 1 페이지 로딩 중...
✅ [유로] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [유로] 2 페이지 로딩 중...
✅ [유로] 2 페이지 완료. 수집된 ID 1개, 새로운 ID 1개 추가.
-> [유로] 3 페이지 로딩 중...
❌ [유로] 3 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandleVerifier [0x0x7

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20012", "타일바스"),       # 예시: 침대 카테고리
    ("20013", "판넬바스")        # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_바스_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 2개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 타일바스 (ID: 20012)
-> [타일바스] 1 페이지 로딩 중...
✅ [타일바스] 1 페이지 완료. 수집된 ID 7개, 새로운 ID 7개 추가.
-> [타일바스] 2 페이지 로딩 중...
❌ [타일바스] 2 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandleVerifier [0x0x7ff77cd5b119+117673]
	GetHandleVerifier [0x0x7ff77cd410a8

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20014", "붙박이장"),       # 예시: 침대 카테고리
    ("20015", "현관수납"),
    ("20016", "공간수납")        # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_수납_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 3개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 붙박이장 (ID: 20014)
-> [붙박이장] 1 페이지 로딩 중...
✅ [붙박이장] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [붙박이장] 2 페이지 로딩 중...
✅ [붙박이장] 2 페이지 완료. 수집된 ID 10개, 새로운 ID 10개 추가.
-> [붙박이장] 3 페이지 로딩 중...
❌ [붙박이장] 3 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandl

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20017", "중문"),       # 예시: 침대 카테고리
    ("20018", "도어")      # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_중문,도어_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 2개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 중문 (ID: 20017)
-> [중문] 1 페이지 로딩 중...
✅ [중문] 1 페이지 완료. 수집된 ID 20개, 새로운 ID 20개 추가.
-> [중문] 2 페이지 로딩 중...
✅ [중문] 2 페이지 완료. 수집된 ID 2개, 새로운 ID 2개 추가.
-> [중문] 3 페이지 로딩 중...
❌ [중문] 3 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandleVerifier [0x0x7

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait # Explicit Wait을 위한 모듈 추가
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By 모듈 추가
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os 
import re # 정규표현식 모듈 (ID 추출용)

# ===============================================
# ⚠️주의 사항: 여기에 실제 한샘몰 클래스 이름을 넣어주세요!
# ===============================================

# 1. 상세 페이지에서 상품명을 감싸는 div 태그의 클래스 (끝 부분)
NAME_CLASS_PART = 'eXsFpn' 

# 2. 상세 페이지에서 가격을 감싸는 div 태그의 클래스 (끝 부분)
PRICE_CLASS_PART = 'gkbSol' 

# 3. 상세 페이지에서 메인 상품 이미지를 감싸는 div/img 태그의 클래스 (선택 사항)
IMAGE_CLASS_PART = '' 

# ===============================================
# 크롤링 설정
# ===============================================
BASE_URL = "https://store.hanssem.com"

# 💡 핵심 변경: 카테고리 ID와 이름을 직접 리스트 형태로 정의합니다.
# (ID, 카테고리 이름) 형식으로 원하는 카테고리를 추가/수정하세요.
CATEGORY_LIST = [
    ("20019", "발코니창"),       # 예시: 침대 카테고리
    ("20020", "기능창")      # 예시: 매트리스 카테고리
    # ("새로운 ID", "새로운 카테고리명"), # 여기에 추가하세요
] 

MAX_PAGES = 30 # 카테고리당 최대 탐색 페이지 수
CRAWL_DELAY = 2 # 요청 사이의 딜레이 (초)를 2초로 늘려 안정성 강화
EXPLICIT_WAIT_TIME = 5 # Selenium 명시적 대기 시간 (최대 10초)

all_product_data = []
all_product_ids = set() # 전체 상품 ID 유일성 보장
product_id_to_categories = {} # {goods_id: ["침대", "수납"], ...} 카테고리 매핑 딕셔너리

# 0단계: 카테고리 ID 자동 추출 기능 삭제됨

# ===============================================
# 1단계: Selenium을 사용하여 모든 상품 ID 수집 (카테고리별 반복)
# ===============================================

print("=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===")

# Chrome WebDriver 초기화
try:
    driver = webdriver.Chrome() 
except Exception as e:
    print(f"❌ WebDriver 초기화 실패. Chrome 드라이버가 설치되어 있는지 확인하세요: {e}")
    exit()

# 💡 수동 입력된 CATEGORY_LIST 확인
if not CATEGORY_LIST:
    print("❌ CATEGORY_LIST에 크롤링할 카테고리 ID가 정의되어 있지 않습니다. 리스트를 채워주세요.")
    driver.quit()
    exit()
print(f"✅ 총 {len(CATEGORY_LIST)}개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.")


# 💡 외부 for 루프: 수동 정의된 카테고리 리스트를 순회합니다.
for category_id, category_name in CATEGORY_LIST:
    print(f"\n[새 카테고리 시작] 카테고리명: {category_name} (ID: {category_id})")
    
    for page_num in range(1, MAX_PAGES + 1):
        page_url = f"{BASE_URL}/category/{category_id}?page={page_num}"
        driver.get(page_url)
        
        print(f"-> [{category_name}] {page_num} 페이지 로딩 중...")
        
        try:
            WebDriverWait(driver, EXPLICIT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-goods-id]'))
            )
            
            # 지연 로딩 해결: 페이지를 스크롤하여 모든 상품을 로드합니다.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 

            html_doc = driver.page_source
            bs = BeautifulSoup(html_doc, 'html.parser')

            product_divs = bs.find_all('div', attrs={'data-goods-id': True}) 
            
            if not product_divs:
                print(f"➡️ [{category_name}] {page_num} 페이지에서 상품 ID를 찾을 수 없습니다. (카테고리 탐색 종료)")
                break
            
            # ID 추출 및 카테고리 정보 저장
            count_new_ids = 0
            for div in product_divs:
                goods_id = div.get('data-goods-id') 
                if goods_id:
                    if goods_id not in product_id_to_categories:
                        product_id_to_categories[goods_id] = []
                        all_product_ids.add(goods_id)
                        count_new_ids += 1
                    
                    # 해당 상품 ID가 속한 카테고리 이름을 딕셔너리에 추가합니다.
                    if category_name not in product_id_to_categories[goods_id]:
                        product_id_to_categories[goods_id].append(category_name)
                        
            print(f"✅ [{category_name}] {page_num} 페이지 완료. 수집된 ID {len(product_divs)}개, 새로운 ID {count_new_ids}개 추가.")
            
        except Exception as e:
            print(f"❌ [{category_name}] {page_num} 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: {e}). 카테고리 탐색 종료.")
            break

        time.sleep(CRAWL_DELAY) 

# WebDriver 닫기
driver.quit()
print(f"\n=== 1단계 요약: 총 {len(all_product_ids)}개의 고유 상품 ID 수집 완료. ===")

# ===============================================
# 2단계: Requests를 사용하여 상세 정보 크롤링
# ===============================================

print("\n=== 2단계: Requests로 상세 정보 크롤링 시작 ===")
total_ids = len(all_product_ids)

for index, goods_id in enumerate(all_product_ids, 1):
    full_detail_url = f"{BASE_URL}/goods/{goods_id}"
    
    # 💡 1단계에서 수집한 카테고리 정보를 가져옵니다.
    categories = product_id_to_categories.get(goods_id, ["카테고리 정보 없음"])
    current_category_name = ", ".join(categories) # 쉼표로 카테고리 이름을 연결합니다.

    print(f"[{index}/{total_ids}] 상세 정보 크롤링 중 ({current_category_name}): {full_detail_url}")
    
    try:
        detail_response = requests.get(full_detail_url, timeout=10)
        detail_response.raise_for_status() 
        
        detail_bs = BeautifulSoup(detail_response.text, 'html.parser')

        # 1. 상품명 추출
        name_tag = detail_bs.find('div', class_=lambda c: c and NAME_CLASS_PART in c)
        product_name = name_tag.get_text(strip=True) if name_tag else "N/A"

        # 2. 가격 추출
        price_tag = detail_bs.find('div', class_=lambda c: c and PRICE_CLASS_PART in c)
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # 가격 텍스트 정리
        try:
            price = int(price_text.replace(',', '').replace('원', '').strip())
        except ValueError:
            price = price_text

        # 3. 이미지 URL 추출
        image_tag = detail_bs.find('img', src=lambda src: src and goods_id in src)
        if not image_tag and IMAGE_CLASS_PART:
             image_tag = detail_bs.find('img', class_=lambda c: c and IMAGE_CLASS_PART in c)

        product_image_url = image_tag.get('src') if image_tag and image_tag.get('src') else "N/A"
        
        # 4. 💡 상품 상세 설명 (부가 설명) 추출 로직 변경
        
        # 💡 변경된 핵심 로직: 'cont-txt' 클래스를 가진 모든 div를 찾습니다.
        description_containers = detail_bs.find_all('div', class_='cont-txt')
        
        all_description_texts = []
        for container in description_containers:
            # 각 컨테이너 내부의 모든 텍스트를 추출하고 공백으로 연결합니다.
            # get_text(separator=' ', strip=True)를 사용하여 <p>나 <span> 태그 사이의 텍스트도 자연스럽게 연결됩니다.
            text_part = container.get_text(separator=' ', strip=True)
            if text_part:
                all_description_texts.append(text_part)

        # 모든 텍스트 부분을 두 칸 공백 ('  ')으로 연결하여 최종 설명 문자열을 만듭니다.
        # 공백 두 칸을 사용하면 각 블록(div)의 구분이 명확해집니다.
        product_description = "  ".join(all_description_texts).strip()

        # 최종적으로 추출된 설명이 없는 경우 처리
        if not product_description:
            product_description = "N/A"

        all_product_data.append({
            '상품ID': goods_id,
            '카테고리명': current_category_name, 
            '상품명': product_name,
            '가격': price,
            'Image URL': product_image_url,
            '부가_설명': product_description # 💡 새로운 필드 추가
        })
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 상세 정보 요청 중 HTTP/네트워크 오류 발생 ({full_detail_url}): {e}")
    except Exception as e:
        print(f"❌ 데이터 추출 중 오류 발생 ({full_detail_url}): {e}")
        
    time.sleep(CRAWL_DELAY) 

# ===============================================
# 3단계: 결과 정리 및 출력
# ===============================================

if all_product_data:
    df = pd.DataFrame(all_product_data)

    print("\n=== 크롤링 결과 미리보기 (상위 5개) ===")
    print(df.head())

    # CSV 파일로 저장
    file_path = 'hanssem_창호_categories_data.csv'
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 총 {len(df)}개의 상품 데이터가 '{file_path}' 파일로 저장되었습니다.")
else:
    print("\n데이터를 수집하지 못했습니다. 클래스 이름과 카테고리 ID를 확인해 주세요.")


=== 1단계: Selenium으로 동적 상품 ID 수집 시작 ===
✅ 총 2개의 카테고리 ID를 수동으로 설정했습니다. 수집을 시작합니다.

[새 카테고리 시작] 카테고리명: 발코니창 (ID: 20019)
-> [발코니창] 1 페이지 로딩 중...
✅ [발코니창] 1 페이지 완료. 수집된 ID 2개, 새로운 ID 2개 추가.
-> [발코니창] 2 페이지 로딩 중...
❌ [발코니창] 2 페이지 로딩 실패 또는 상품을 찾지 못함 (오류: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff77cd51eb5+80197]
	GetHandleVerifier [0x0x7ff77cd51f10+80288]
	(No symbol) [0x0x7ff77cad02fa]
	(No symbol) [0x0x7ff77cb27cd7]
	(No symbol) [0x0x7ff77cb27f9c]
	(No symbol) [0x0x7ff77cb7ba87]
	(No symbol) [0x0x7ff77cb503bf]
	(No symbol) [0x0x7ff77cb787fb]
	(No symbol) [0x0x7ff77cb50153]
	(No symbol) [0x0x7ff77cb18b02]
	(No symbol) [0x0x7ff77cb198d3]
	GetHandleVerifier [0x0x7ff77d00e83d+2949837]
	GetHandleVerifier [0x0x7ff77d008c6a+2926330]
	GetHandleVerifier [0x0x7ff77d0286c7+3055959]
	GetHandleVerifier [0x0x7ff77cd6cfee+191102]
	GetHandleVerifier [0x0x7ff77cd750af+224063]
	GetHandleVerifier [0x0x7ff77cd5af64+117236]
	GetHandleVerifier [0x0x7ff77cd5b119+117673]
	GetHandleVerifier [0x0x7ff77cd410a8